In [1]:
#SH
import yfinance as yf #Allows us to Use Yahoo Finance API 
import pandas as pd
import numpy as np
import json

In [2]:
#SH
#To setup the postgreSQL Database using PgAdmin

import psycopg2, os

print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    database="postgres",
    user="postgres",
    password="123")
cur = conn.cursor()

Connecting to the PostgreSQL database...


In [3]:
#SH
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

#CREATE THE TABLES TO BE USED IN THE DATABASE
createCmd = """
DROP TABLE IF EXISTS stock_prices;
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS trades;

CREATE TABLE stock_prices (
                price_date date,
                ticker varchar(20) NOT NULL,
                price float NOT NULL,
                PRIMARY KEY (price_date, ticker)
                );
                
                CREATE TABLE trades (
                uid integer NOT NULL,
                ticker varchar(20) NOT NULL,
                trade_date date,
                amount integer
                );
        
                
                CREATE TABLE users (
                uid integer,
                user_name varchar (100),
                fantasy_name varchar (100),
                PRIMARY KEY (uid)
                );
            """
    
cur.execute(createCmd)
conn.commit()

PostgreSQL database version:
('PostgreSQL 13.4 (Debian 13.4-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [4]:
#SH
#Specifying the Stock tickers to be pulled from the API
array = ["MSFT", "AAPL", "TSLA", "SONY", "MCD", "FB", "JPM", "DIS", "COST", "CVS", "NFLX", "SBUX", "UBER", 
         "NKE", "AMZN", "PFE", "GOOGL", "LRLCY", "DELL", "UNH", "SPY", "VTI", "VOO", "VWO", "VEA", "QQQ", 
         "IVV", "IEFA", "EFA", "AGG"]

#Loop to Pull the Data from the API
prices = []
for ticker in array:
    open_prices = yf.Ticker(ticker).history(period="6mo").filter(["Open"])
    open_prices.reset_index()
    open_prices["date"] = open_prices.index
    open_prices["ticker"] = ticker
    prices.extend(open_prices[["date", "ticker", "Open"]].values)
print(len(prices), "prices")


3840 prices


In [5]:
#SH
#Inserts the Data into the Table in PgAdmin SQL 
print(len(prices), "prices")
query = """INSERT INTO stock_prices(price_date,ticker,price) VALUES(%s, %s, %s)"""
cur.executemany(query, tuple(prices))
conn.commit()

3840 prices


In [6]:
#SH
#Pulling the Trade Data
trade_data = pd.read_csv("User_Trades.csv", skipinitialspace=True)
print(len(trade_data.values), "trades")

600 trades


In [28]:
#SH
trade_data

,uid,ticker,trade_date,amount
0,2,PFE,12/12/2021,-3
1,4,JPM,12/11/2021,23
2,2,AAPL,10/22/2021,-10
3,4,GOOGL,9/7/2021,-10
4,5,SBUX,9/18/2021,18
...,...,...,...,...
595,2,SBUX,9/14/2021,-16
596,6,SPY,9/9/2021,-9
597,5,NKE,10/5/2021,17
598,2,COST,11/7/2021,1


In [8]:
#SH
#Inserting the Trade Data into PgAdmin SQL 
print(len(trade_data), 'trades')
query = """INSERT INTO trades(uid,ticker,trade_date,amount) VALUES(%s, %s, %s, %s)"""
cur.executemany(query, trade_data.values)
conn.commit()

600 trades


In [9]:
#SH
# Filter data to only trades that have a price. The rest are bad/illegal trades that cannot be executed as they were done on days where you cannot trade
query = """
DROP TABLE IF EXISTS trades_clean;

select t.* 
into trades_clean
from trades t, stock_prices p
where t.trade_date = p.price_date
and t.ticker = p.ticker;
"""
cur.execute(query)
conn.commit()

In [10]:
#SH
# Pulling the User Data
user_data = pd.read_csv("User_Data.csv")
print(len(user_data.values), "users")

6 users


In [11]:
#SH
user_data

,uid,user_name,fantasy_name
0,1,Sandy He,Never Gonna Give You Up
1,2,Randy Le,Never Gonna Let You Down
2,3,Andy Ye,Never Gonna Run Around and Desert You
3,4,Daisy Wen,Never Gonna Make You Cry
4,5,Maisie Yen,Never Gonna Say Goodbye
5,6,Jacy Sen,Never Gonna Tell A Lie and Hurt You


In [12]:
#SH
#Inserting the User Data into PgAdmin SQL

print(len(user_data.values), "users")
query = """INSERT INTO users(uid, user_name, fantasy_name) VALUES(%s, %s, %s)"""
cur.executemany(query, user_data.values)
conn.commit()

6 users


In [13]:
#SH
#Create the Total Changes Table Which Summarizes the Differences in Stock Balances Based on User Trades 

createCmd = """
DROP TABLE IF EXISTS total_changes;

SELECT uid, SUM(t.amount * p.price) AS final_change
INTO total_changes
FROM trades_clean t, stock_prices p
WHERE t.ticker = p.ticker
AND t.trade_date = p.price_date
GROUP BY uid

"""
    
cur.execute(createCmd)
conn.commit()

In [14]:
#SH
# Read data from PostgreSQL database table and load into a DataFrame instance

df_total_changes = pd.read_sql("SELECT * FROM \"total_changes\"", conn);
 
pd.set_option('display.expand_frame_repr', False);

# Print the DataFrame
df_total_changes

,uid,final_change
0,4,259306.994437
1,6,98231.940708
2,3,44780.312859
3,1,-12281.013532
4,5,-8845.213718
5,2,42778.596075


In [15]:
#SH
#Creates the Final Balances Table Which Summarizes the Total Balance in Stocks Users Still Have in Their Portfolio

createCmd = """
DROP TABLE IF EXISTS final_balances;

SELECT uid, t.ticker, SUM(t.amount* p.price) AS final_balance
INTO final_balances
FROM trades_clean t, stock_prices p
WHERE t.uid = t.uid
AND t.ticker = p.ticker
AND p.price_date = (SELECT MAX(price_date) FROM stock_prices)
GROUP BY uid, t.ticker

"""
    
cur.execute(createCmd)
conn.commit()

In [16]:
#SH
# Read data from PostgreSQL database table and load into a DataFrame instance

df_final_balance = pd.read_sql("SELECT * FROM \"final_balances\"", conn);
 
pd.set_option('display.expand_frame_repr', False);

# Print the DataFrame
df_final_balance

,uid,ticker,final_balance
0,4,QQQ,3833.299866
1,1,IVV,14369.119659
2,6,NKE,-3394.230103
3,3,AMZN,-50313.149414
4,2,COST,9887.760132
...,...,...,...
156,4,GOOGL,114000.000000
157,1,QQQ,5749.949799
158,1,VEA,-1873.679977
159,2,MSFT,4171.440063


In [17]:
#SH
#Combining the Balance and the Changes to Get the Final Portfolio Amount

createCmd = """
DROP TABLE IF EXISTS final_summary;

SELECT u.user_name, u.fantasy_name, final_change, fb
INTO final_summary
FROM users u, (SELECT uid, SUM(final_balance) as fb 
FROM final_balances f GROUP BY uid) fb, total_changes t
WHERE t.uid = u.uid
AND u.uid = fb.uid
"""
    
cur.execute(createCmd)
conn.commit()

In [18]:
#SH
#To Check All The Trades Made By One User
#To Change Which User just update "WHERE uid =" to user you want to select

df_trades_user_1 = pd.read_sql("""
SELECT uid, ticker, amount
FROM trades_clean 
WHERE uid = 1 
GROUP BY uid, ticker, amount
""", conn);

# Print the DataFrame
df_trades_user_1

,uid,ticker,amount
0,1,AAPL,-18
1,1,AMZN,-14
2,1,COST,8
3,1,CVS,-8
4,1,DELL,22
5,1,FB,-20
6,1,IVV,-20
7,1,IVV,15
8,1,IVV,21
9,1,JPM,-18


In [19]:
#SH
#To Check The Trade Balance Made By One User
#To Change Which User just update "WHERE uid =" to user you want to select

df_trades_user_1 = pd.read_sql("""
SELECT uid, ticker, sum(amount) FROM \"trades_clean\" WHERE uid = 1 GROUP BY uid, ticker
""", conn);

# Print the DataFrame
df_trades_user_1

,uid,ticker,sum
0,1,AAPL,-18
1,1,AMZN,-14
2,1,COST,8
3,1,CVS,-8
4,1,DELL,22
5,1,FB,-20
6,1,IVV,31
7,1,JPM,-13
8,1,LRLCY,-4
9,1,MCD,23


In [20]:
#SH
#To Check The Remaining Stocks and Balance For Each User
#To Change which User just change WHERE t.uid = to another uid

df_balance_user_1 = pd.read_sql("""
SELECT uid, t.ticker, SUM(t.amount), p.price, SUM(t.amount* p.price) AS final_balance
FROM trades_clean t, stock_prices p
WHERE t.uid = 1
AND t.ticker = p.ticker
AND p.price_date = (SELECT MAX(price_date) FROM stock_prices)
GROUP BY uid, t.ticker, p.price
""", conn);

# Print the DataFrame
df_balance_user_1

,uid,ticker,sum,price,final_balance
0,1,AAPL,-18,169.929993,-3058.739868
1,1,AMZN,-14,3354.209961,-46958.939453
2,1,COST,8,549.320007,4394.560059
3,1,CVS,-8,101.379997,-811.039978
4,1,DELL,22,54.919998,1208.239960
5,1,FB,-20,332.799988,-6655.999756
6,1,IVV,31,463.519989,14369.119659
7,1,JPM,-13,159.320007,-2071.160095
8,1,LRLCY,-4,92.870003,-371.480011
9,1,MCD,23,264.000000,6072.000000


In [21]:
#SH
#Final Summaries for the Users

df_final_summary = pd.read_sql("SELECT * FROM \"final_summary\" ORDER BY fb - final_change DESC", conn);
 
pd.set_option('display.expand_frame_repr', False);

# Print the DataFrame
df_final_summary

,user_name,fantasy_name,final_change,fb
0,Maisie Yen,Never Gonna Say Goodbye,-8845.213718,2862.990986
1,Andy Ye,Never Gonna Run Around and Desert You,44780.312859,55136.361809
2,Sandy He,Never Gonna Give You Up,-12281.013532,-7098.429287
3,Jacy Sen,Never Gonna Tell A Lie and Hurt You,98231.940708,99363.838665
4,Randy Le,Never Gonna Let You Down,42778.596075,43238.488590
5,Daisy Wen,Never Gonna Make You Cry,259306.994437,258952.517342


In [22]:
#SH
#This is the Final Rankings of the Users Based on Their Final Portfolio Amount

df_final_portfolio_amount = pd.read_sql("SELECT user_name, fantasy_name, fb - final_change as final_portfolio_amount FROM \"final_summary\" ORDER BY final_portfolio_amount DESC", conn);
 
pd.set_option('display.expand_frame_repr', False);

df_final_portfolio_amount

,user_name,fantasy_name,final_portfolio_amount
0,Maisie Yen,Never Gonna Say Goodbye,11708.204704
1,Andy Ye,Never Gonna Run Around and Desert You,10356.048950
2,Sandy He,Never Gonna Give You Up,5182.584245
3,Jacy Sen,Never Gonna Tell A Lie and Hurt You,1131.897957
4,Randy Le,Never Gonna Let You Down,459.892515
5,Daisy Wen,Never Gonna Make You Cry,-354.477096
